### Seria Lab: add a new part to the inventory
Adding new part to a squadron's inventory requires modifying attributes such as `m_id` and `m_master_id`. This is similar to adding ship to a fleet. But require less operations.

In [24]:
from copy import deepcopy
from seria import *

pf_path = 'sample/profile.seria'
pf_node = load(pf_path)

parts_path = 'sample/parts.seria'
parts_node = load(parts_path)

Before we add a new part, we can list out existing parts in inventory. This is done by first locate player's squadron. And find the child node that is defined as the inventory.

In [25]:
# Player's squadron
escadra_node = pf_node.get_node_if(
    lambda n: n.get_attribute('m_name') == 'MARK')

# Player's inventory
inventory_node = escadra_node.get_node_if(
    lambda n: n.header == 'm_inventory=7')

id_inventory = inventory_node.get_attribute('m_id')
print(f"inventory m_id: {id_inventory}")

# list out all the parts in the inventory
print('oid\t\tcount\tm_master_id')

for part in inventory_node.get_nodes():
    oid = part.get_attribute('m_oid')
    count = part.get_attribute('m_count')
    master_id = part.get_attribute('m_master_id')

    print(f'{oid}\t{count}\t{master_id}')

inventory m_id: 691432484788337130
oid		count	m_master_id
MDL_QUARTERS_02	None	691432484788337130
MDL_FSS_02	2	691432484788337130
ITEM_AMMO	2	691432484788337130
ITEM_ARMOR	2	691432484788337130


Based on the output above, we can see that parts with multiple quantities have an attribute `m_count`, while parts with a quantity of one do not have this attribute. Similar to ships and other nodes that follow a hierarchical structure, these parts use the `master_id` attribute to indicate their relationship with the parent node above them. Therefore, when adding a new part, it is crucial to set the `master_id` correctly.
1. Obtain all unique ids from the profile.
2. Retrieve the selected part from the editor library node (`parts.seria`). Make a deep copy of it, so we can reuse the original node multiple times without repeating the parsing.
3. Locate the target squadron's inventory
4. Set part `m_id` to a newly generated id, and update relevant child nodes' attributes accordingly.
5. Set part `m_master_id` to target inventory `m_id`
6. Add part node to inventory node.

In [26]:
def get_unique_ids(seria: SeriaNode) -> set:
    unique_ids = set()

    id = seria.get_attribute('m_id')
    if id:
        unique_ids.add(int(id))

    for child in seria.get_nodes():
        unique_ids.update(get_unique_ids(child))

    return unique_ids

In [ ]:
# step 1
ids = get_unique_ids(pf_node)

# MDL_MISSILE_01 (R-5 ZENITH)
# step 2
missile_node = parts_node.get_node_if(
    lambda n: n.get_attribute('m_oid') == 'MDL_MISSILE_01')
m_node = deepcopy(missile_node)

# step 4
new_id = max(ids) + 1
m_node.update_attribute('m_id', str(new_id))

# step 5
m_node.set_attribute('m_master_id', id_inventory)

# if we want to add multiple missiles (optional)
m_node.put_attribute_after('m_count', '5', 'm_oid')

# step 6
inventory_node.add_node(m_node)

# save the modified profile
pf_path2 = 'sample/profile2.seria'
dump(pf_node, pf_path2)